# Configuration Propagation

Hypster supports nested configurations by providing a `hp.propagate` API to select and override values at different levels.

## Using `hp.propagate()`

The `hp.propagate()` function allows you to include one configuration within another, propagating selections and overrides.

Example:

We first define a hypster config function. It can be in the same Jupyter Notebook/Python Module or in a different one:

In [2]:
import hypster
from hypster import HP

@hypster.config
def my_config(hp: HP):
    llm_model = hp.select({'haiku': 'claude-3-haiku-20240307', 
                           'sonnet': 'claude-3-5-sonnet-20240620',
                           'gpt-4o-mini': 'gpt-4o-mini'}, default='gpt-4o-mini')
    
    llm_config = {'temperature': hp.number_input(0), 
                  'max_tokens': hp.number_input(64)}
    
    system_prompt = hp.text_input('You are a helpful assistant. Answer with one word only')

In [3]:
hypster.save(my_config, "my_config.py")

- We can then `load` it from its path and have it be part of the parent configuration.
- We can select & override values within our nested configuration by using dot notation

In [8]:
from hypster import HP, config

@config
def my_config_parent(hp: HP):
    import hypster
    my_config = hypster.load("my_config.py")
    my_conf = hp.propagate(my_config)
    a = hp.select(["a", "b", "c"], default="a")

my_config_parent(selections={"my_conf.llm_model": "haiku"},
                 overrides={"a" : "d", 
                            "my_conf.system_prompt": "You are a helpful assistant. Answer with *two words* only"})

{'my_config': <hypster.core.Hypster at 0x10ad11390>,
 'my_conf': {'llm_model': 'claude-3-haiku-20240307',
  'llm_config': {'temperature': 0, 'max_tokens': 64},
  'system_prompt': 'You are a helpful assistant. Answer with *two words* only'},
 'a': 'd'}